In [1]:
from google.colab import files
uploaded = files.upload()

Saving test_weather.csv to test_weather (1).csv
Saving train_weather.csv to train_weather (1).csv


In [2]:
import pandas  as pd

from sklearn.preprocessing import LabelEncoder

In [3]:
train = pd.read_csv('train_weather.csv')

In [102]:
test = pd.read_csv('test_weather.csv')

In [4]:
train

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,Brisbane,13.7,23.6,0.0,5.0,9.6,SSE,24.0,SSW,ESE,7.0,9.0,65.0,53.0,1028.4,1026.7,1.0,1.0,18.3,22.3,0,0
1,Sydney,15.4,21.2,61.2,5.8,2.7,S,43.0,WSW,SSW,19.0,24.0,91.0,65.0,1018.8,1016.5,8.0,7.0,15.6,20.7,1,1
2,Richmond,16.8,30.9,1.0,NaN,NaN,NE,35.0,NNW,NE,7.0,26.0,80.0,38.0,1020.4,1013.2,1.0,NaN,19.6,30.0,0,0
3,NorfolkIsland,13.9,17.1,0.2,3.8,8.9,SE,37.0,SE,SSE,17.0,20.0,56.0,63.0,1027.7,1026.1,3.0,1.0,15.3,15.8,0,0
4,Mildura,16.0,36.1,0.0,6.8,12.9,ENE,33.0,ENE,E,17.0,19.0,58.0,32.0,1018.5,1016.0,0.0,3.0,26.0,34.9,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121868,GoldCoast,17.6,25.1,0.0,NaN,NaN,SSE,56.0,SE,E,9.0,19.0,58.0,58.0,1012.8,1010.2,NaN,NaN,23.5,24.2,0,0
121869,Adelaide,15.1,25.2,0.0,10.0,11.3,SW,48.0,S,WSW,11.0,28.0,55.0,41.0,1018.8,1018.3,NaN,NaN,19.0,24.6,0,0
121870,MountGinini,10.5,18.4,1.2,NaN,NaN,SSE,50.0,E,E,15.0,13.0,97.0,93.0,NaN,NaN,NaN,NaN,12.5,15.9,1,1
121871,Adelaide,13.6,19.8,2.6,NaN,7.5,WSW,54.0,WSW,WSW,11.0,20.0,76.0,46.0,1008.5,1007.4,NaN,NaN,14.4,18.9,1,1


In [5]:
print('\n-----------------РАЗМЕРНОСТЬ И ТИП ДАННЫХ---------------------------\n')
print(train.info())
print('\n------------------------НАЛИЧИЕ ПУСТЫХ ЗНАЧЕНИЙ-----------------------------\n')
print(train.isna().sum())
print('\n------------------------БАЛАНС КЛАССОВ-----------------------------\n')
print('RainTomorrow == 0:\t',train[train['RainTomorrow'] == 0].shape[0])
print('RainTomorrow == 1:\t',train[train['RainTomorrow'] == 1].shape[0])


-----------------РАЗМЕРНОСТЬ И ТИП ДАННЫХ---------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121873 entries, 0 to 121872
Data columns (total 22 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Location       121873 non-null  object 
 1   MinTemp        120652 non-null  float64
 2   MaxTemp        120842 non-null  float64
 3   Rainfall       119145 non-null  float64
 4   Evaporation    71329 non-null   float64
 5   Sunshine       65698 non-null   float64
 6   WindGustDir    113157 non-null  object 
 7   WindGustSpeed  113212 non-null  float64
 8   WindDir9am     112959 non-null  object 
 9   WindDir3pm     118519 non-null  object 
 10  WindSpeed9am   120354 non-null  float64
 11  WindSpeed3pm   119528 non-null  float64
 12  Humidity9am    119642 non-null  float64
 13  Humidity3pm    118491 non-null  float64
 14  Pressure9am    109238 non-null  float64
 15  Pressure3pm    109271 non-null  float64
 16  Clo

In [6]:
# РАБОТА С ПУСТЫМИ ЗНАЧЕНИЯМИ

# пустые значения есть почти во всех колонках, кроме Location, RainToday, RainTomorrow
# в колонках Evaporation, Sunshine, Cloud9am, Cloud3pm кол-во nan близко в к половине кол-ва всех данных, поэтому их целесообразно просто удалить
# в остальных случаях заполнить средними значениями по столбцу с учетом значения в колонке Location (кроме WindGustDir, WindDir9am, WindDir3pm тк тут категориальные признаки)

In [7]:
train = train.drop(['Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm'], axis = 1)

In [8]:
locations = ['Brisbane', 'Sydney', 'Richmond', 'NorfolkIsland', 'Mildura',
       'Adelaide', 'GoldCoast', 'Woomera', 'Witchcliffe', 'Uluru',
       'Cairns', 'Bendigo', 'Newcastle', 'Moree', 'Canberra',
       'Williamtown', 'PerthAirport', 'Katherine', 'Albury', 'Nhil',
       'Perth', 'Melbourne', 'Townsville', 'WaggaWagga', 'MountGinini',
       'Ballarat', 'Watsonia', 'Penrith', 'NorahHead', 'Portland',
       'SalmonGums', 'BadgerysCreek', 'MelbourneAirport', 'Walpole',
       'Hobart', 'CoffsHarbour', 'SydneyAirport', 'Albany', 'Tuggeranong',
       'Nuriootpa', 'AliceSprings', 'Wollongong', 'PearceRAAF', 'Cobar',
       'Dartmoor', 'Darwin', 'MountGambier', 'Sale', 'Launceston']

columns_nan_train = ['MinTemp', 'MaxTemp', 'Rainfall', 'WindGustSpeed', 'WindSpeed9am',
       'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am','Pressure3pm', 'Temp9am', 'Temp3pm']

for loc in locations:
    average_data = []

    for col in columns_nan_train:
        average_data.append(train[train['Location'] == loc][col].mean())
    d = dict(zip(columns_nan_train, average_data))
    train[train['Location'] == loc] = train[train['Location'] == loc].fillna(value = d)

In [9]:
print(train.isna().sum())

Location             0
MinTemp              0
MaxTemp              0
Rainfall             0
WindGustDir       8716
WindGustSpeed     5103
WindDir9am        8914
WindDir3pm        3354
WindSpeed9am         0
WindSpeed3pm         0
Humidity9am          0
Humidity3pm          0
Pressure9am      10211
Pressure3pm      10211
Temp9am              0
Temp3pm              0
RainToday            0
RainTomorrow         0
dtype: int64


In [ ]:
# в некоторых случаях значения в столбцах, относящиеся к определенному городу полностью пустые
# тогда заполним их средним значением по столбцу
# 'WindGustSpeed', 'Pressure9am', 'Pressure3pm'

In [10]:
train['WindGustSpeed'] = train['WindGustSpeed'].fillna(train['WindGustSpeed'].mean())
train['Pressure9am'] = train['Pressure9am'].fillna(train['Pressure9am'].mean())
train['Pressure3pm'] = train['Pressure3pm'].fillna(train['Pressure3pm'].mean())

In [11]:
print(train.isna().sum())

Location            0
MinTemp             0
MaxTemp             0
Rainfall            0
WindGustDir      8716
WindGustSpeed       0
WindDir9am       8914
WindDir3pm       3354
WindSpeed9am        0
WindSpeed3pm        0
Humidity9am         0
Humidity3pm         0
Pressure9am         0
Pressure3pm         0
Temp9am             0
Temp3pm             0
RainToday           0
RainTomorrow        0
dtype: int64


In [ ]:
# аналогично работаем со столбцами  WindGustDir, WindDir9am, WindDir3pm  с категориальными признакми, но где есть и nan,
# только заполнять их будем самым распространенным значением, вместо среднего

In [12]:
for loc in locations:
    average_data = []

    for col in 'WindGustDir', 'WindDir9am', 'WindDir3pm':
        unik = train[train['Location'] == loc][col].unique()
        v_c = train[train['Location'] == loc][col].value_counts(sort=False)

        maks = 0
        for i in range(len(v_c)):
            if v_c[i] > v_c[maks]:
                maks = i
    
        average_data.append(unik[maks])
    d = dict(zip(['WindGustDir', 'WindDir9am', 'WindDir3pm'], average_data))
    train[train['Location'] == loc] = train[train['Location'] == loc].fillna(value = d)

In [13]:
train.isna().sum()

Location            0
MinTemp             0
MaxTemp             0
Rainfall            0
WindGustDir      5180
WindGustSpeed       0
WindDir9am          0
WindDir3pm       1008
WindSpeed9am        0
WindSpeed3pm        0
Humidity9am         0
Humidity3pm         0
Pressure9am         0
Pressure3pm         0
Temp9am             0
Temp3pm             0
RainToday           0
RainTomorrow        0
dtype: int64

In [14]:
l = []
for i in 'WindGustDir', 'WindDir3pm':
    unik = train['WindGustDir'].unique()
    v_c = train['WindGustDir'].value_counts(sort=False)
    maks = 0
    for i in range(len(v_c)):
        if v_c[i] > v_c[maks]:
            maks = i
    
    l.append(unik[maks])

train['WindGustDir'] = train['WindGustDir'].fillna(l[0])
train['WindDir3pm'] = train['WindDir3pm'].fillna(l[1])

In [15]:
train.isna().sum()

Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64

In [ ]:
# РАБОТА С КАТЕГОРИАЛЬНЫМИ ПРИЗНАКАМИ
# тк в столбцах с категориальными признаками: Location, WindGustDir, WindDir9am, WindDir3pm, слишком много уникальных значений(49, 17, 17, 17),
# поэтому лучше всего будет закодировать данные с помощью LabelEncoder 

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121873 entries, 0 to 121872
Data columns (total 18 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Location       121873 non-null  object 
 1   MinTemp        121873 non-null  float64
 2   MaxTemp        121873 non-null  float64
 3   Rainfall       121873 non-null  float64
 4   WindGustDir    121873 non-null  object 
 5   WindGustSpeed  121873 non-null  float64
 6   WindDir9am     121873 non-null  object 
 7   WindDir3pm     121873 non-null  object 
 8   WindSpeed9am   121873 non-null  float64
 9   WindSpeed3pm   121873 non-null  float64
 10  Humidity9am    121873 non-null  float64
 11  Humidity3pm    121873 non-null  float64
 12  Pressure9am    121873 non-null  float64
 13  Pressure3pm    121873 non-null  float64
 14  Temp9am        121873 non-null  float64
 15  Temp3pm        121873 non-null  float64
 16  RainToday      121873 non-null  int64  
 17  RainTomorrow   121873 non-nul

In [17]:
encoder = LabelEncoder()

for col in 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm':
    label = encoder.fit_transform(train[col])
    train[col] = label

In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121873 entries, 0 to 121872
Data columns (total 18 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Location       121873 non-null  int64  
 1   MinTemp        121873 non-null  float64
 2   MaxTemp        121873 non-null  float64
 3   Rainfall       121873 non-null  float64
 4   WindGustDir    121873 non-null  int64  
 5   WindGustSpeed  121873 non-null  float64
 6   WindDir9am     121873 non-null  int64  
 7   WindDir3pm     121873 non-null  int64  
 8   WindSpeed9am   121873 non-null  float64
 9   WindSpeed3pm   121873 non-null  float64
 10  Humidity9am    121873 non-null  float64
 11  Humidity3pm    121873 non-null  float64
 12  Pressure9am    121873 non-null  float64
 13  Pressure3pm    121873 non-null  float64
 14  Temp9am        121873 non-null  float64
 15  Temp3pm        121873 non-null  float64
 16  RainToday      121873 non-null  int64  
 17  RainTomorrow   121873 non-nul

In [19]:
train

,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,7,13.7,23.6,0.0,10,24.0,11,2,7.0,9.0,65.0,53.0,1028.400000,1026.700000,18.3,22.3,0,0
1,37,15.4,21.2,61.2,8,43.0,15,11,19.0,24.0,91.0,65.0,1018.800000,1016.500000,15.6,20.7,1,1
2,34,16.8,30.9,1.0,4,35.0,6,4,7.0,26.0,80.0,38.0,1020.400000,1013.200000,19.6,30.0,0,0
3,27,13.9,17.1,0.2,9,37.0,9,10,17.0,20.0,56.0,63.0,1027.700000,1026.100000,15.3,15.8,0,0
4,20,16.0,36.1,0.0,1,33.0,1,0,17.0,19.0,58.0,32.0,1018.500000,1016.000000,26.0,34.9,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121868,14,17.6,25.1,0.0,10,56.0,9,0,9.0,19.0,58.0,58.0,1012.800000,1010.200000,23.5,24.2,0,0
121869,0,15.1,25.2,0.0,12,48.0,8,15,11.0,28.0,55.0,41.0,1018.800000,1018.300000,19.0,24.6,0,0
121870,23,10.5,18.4,1.2,10,50.0,0,0,15.0,13.0,97.0,93.0,1017.662795,1015.272861,12.5,15.9,1,1
121871,0,13.6,19.8,2.6,15,54.0,15,15,11.0,20.0,76.0,46.0,1008.500000,1007.400000,14.4,18.9,1,1


In [20]:
X = train.drop(['RainTomorrow'], axis = 1)
y = train['RainTomorrow']

In [21]:
!pip install catboost

In [34]:
from sklearn.metrics import classification_report

from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import  KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

In [23]:
models = [CatBoostClassifier(), DecisionTreeClassifier(), XGBClassifier(), LogisticRegression(), KNeighborsClassifier(), RandomForestClassifier()]
models_names = ['CatBoostClassifier', 'DecisionTreeClassifier', 'XGBClassifier', 'LogisticRegression', 'KNeighborsClassifier', 'RandomForestClassifier']

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 43)

In [25]:
for i in range(len(models)):
    model = models[i]
    model_name = models_names[i]

    model.fit(X_train, y_train)
    predicted = model.predict(X_test)

    print('\n____________________', model_name, '_______________________\n')
    print(classification_report(y_test, predicted))

Learning rate set to 0.072817
0:	learn: 0.6434449	total: 86.7ms	remaining: 1m 26s
1:	learn: 0.6001404	total: 113ms	remaining: 56.3s
2:	learn: 0.5650636	total: 136ms	remaining: 45.1s
3:	learn: 0.5355298	total: 161ms	remaining: 40.1s
4:	learn: 0.5112906	total: 186ms	remaining: 37s
5:	learn: 0.4914598	total: 219ms	remaining: 36.3s
6:	learn: 0.4739885	total: 249ms	remaining: 35.3s
7:	learn: 0.4606645	total: 276ms	remaining: 34.3s
8:	learn: 0.4484917	total: 302ms	remaining: 33.2s
9:	learn: 0.4379938	total: 327ms	remaining: 32.4s
10:	learn: 0.4285278	total: 354ms	remaining: 31.8s
11:	learn: 0.4212893	total: 380ms	remaining: 31.2s
12:	learn: 0.4147048	total: 406ms	remaining: 30.8s
13:	learn: 0.4087742	total: 439ms	remaining: 30.9s
14:	learn: 0.4037510	total: 464ms	remaining: 30.5s
15:	learn: 0.3994909	total: 491ms	remaining: 30.2s
16:	learn: 0.3962984	total: 516ms	remaining: 29.9s
17:	learn: 0.3929614	total: 543ms	remaining: 29.6s
18:	learn: 0.3905901	total: 569ms	remaining: 29.4s
19:	learn: 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,



____________________ LogisticRegression _______________________

              precision    recall  f1-score   support

           0       0.86      0.95      0.90     19068
           1       0.70      0.43      0.54      5307

    accuracy                           0.84     24375
   macro avg       0.78      0.69      0.72     24375
weighted avg       0.82      0.84      0.82     24375


____________________ KNeighborsClassifier _______________________

              precision    recall  f1-score   support

           0       0.87      0.94      0.90     19068
           1       0.68      0.49      0.57      5307

    accuracy                           0.84     24375
   macro avg       0.77      0.71      0.74     24375
weighted avg       0.83      0.84      0.83     24375


____________________ RandomForestClassifier _______________________

              precision    recall  f1-score   support

           0       0.87      0.95      0.91     19068
           1       0.74      0.49

In [ ]:
# попробуем уулчшить оценки с помощью балансировки классов

In [61]:
df_train = train[train['RainTomorrow'] == 1].reset_index().drop(['index'], axis = 1)

In [71]:
df = train[train['RainTomorrow'] == 0].reset_index().drop(['index'], axis = 1)

In [73]:
loc_count = [0]*49

In [75]:
for i in range(len(df)):
    if loc_count[df['Location'][i]] < 650:
        l = []
        for x in df_train.columns:
            l.append(df[x][i])
        df_train.loc[len(df_train)] = l
        loc_count[df['Location'][i]] += 1


In [81]:
df_train[df_train['RainTomorrow'] == 1].shape[0]

26858

In [82]:
df_train[df_train['RainTomorrow'] == 0].shape[0]

31853

In [80]:
X_new = df_train.drop(['RainTomorrow'], axis = 1)
y = df_train['RainTomorrow']

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state = 43)

In [85]:
for i in range(len(models)):
    model = models[i]
    model_name = models_names[i]

    model.fit(X_train, y_train)
    predicted = model.predict(X_test)

    print('\n____________________', model_name, '_______________________\n')
    print(classification_report(y_test, predicted))

Learning rate set to 0.053308
0:	learn: 0.6736442	total: 14.9ms	remaining: 14.9s
1:	learn: 0.6533222	total: 29.3ms	remaining: 14.6s
2:	learn: 0.6361157	total: 44.8ms	remaining: 14.9s
3:	learn: 0.6213311	total: 59.5ms	remaining: 14.8s
4:	learn: 0.6081470	total: 76.7ms	remaining: 15.3s
5:	learn: 0.5962361	total: 91.1ms	remaining: 15.1s
6:	learn: 0.5863260	total: 106ms	remaining: 15.1s
7:	learn: 0.5769234	total: 121ms	remaining: 15s
8:	learn: 0.5692668	total: 135ms	remaining: 14.9s
9:	learn: 0.5613625	total: 150ms	remaining: 14.8s
10:	learn: 0.5540924	total: 165ms	remaining: 14.8s
11:	learn: 0.5475605	total: 179ms	remaining: 14.8s
12:	learn: 0.5417969	total: 194ms	remaining: 14.7s
13:	learn: 0.5366453	total: 212ms	remaining: 15s
14:	learn: 0.5321241	total: 228ms	remaining: 15s
15:	learn: 0.5282792	total: 244ms	remaining: 15s
16:	learn: 0.5242440	total: 263ms	remaining: 15.2s
17:	learn: 0.5206354	total: 282ms	remaining: 15.4s
18:	learn: 0.5172703	total: 297ms	remaining: 15.3s
19:	learn: 0.

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,



____________________ LogisticRegression _______________________

              precision    recall  f1-score   support

         0.0       0.75      0.81      0.78      6275
         1.0       0.76      0.69      0.72      5468

    accuracy                           0.76     11743
   macro avg       0.76      0.75      0.75     11743
weighted avg       0.76      0.76      0.75     11743


____________________ KNeighborsClassifier _______________________

              precision    recall  f1-score   support

         0.0       0.75      0.81      0.78      6275
         1.0       0.76      0.70      0.73      5468

    accuracy                           0.76     11743
   macro avg       0.76      0.75      0.76     11743
weighted avg       0.76      0.76      0.76     11743


____________________ RandomForestClassifier _______________________

              precision    recall  f1-score   support

         0.0       0.79      0.84      0.81      6275
         1.0       0.80      0.74

In [ ]:
# оценки моделей после балансировки классов оценки ухудшились

In [87]:
X = train.drop(['RainTomorrow'], axis = 1)
y = train['RainTomorrow']

In [88]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 43)

In [89]:
model = CatBoostClassifier()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

Learning rate set to 0.072817
0:	learn: 0.6434449	total: 71.2ms	remaining: 1m 11s
1:	learn: 0.6001404	total: 141ms	remaining: 1m 10s
2:	learn: 0.5650636	total: 205ms	remaining: 1m 8s
3:	learn: 0.5355298	total: 281ms	remaining: 1m 9s
4:	learn: 0.5112906	total: 353ms	remaining: 1m 10s
5:	learn: 0.4914598	total: 446ms	remaining: 1m 13s
6:	learn: 0.4739885	total: 529ms	remaining: 1m 15s
7:	learn: 0.4606645	total: 597ms	remaining: 1m 14s
8:	learn: 0.4484917	total: 636ms	remaining: 1m 10s
9:	learn: 0.4379938	total: 680ms	remaining: 1m 7s
10:	learn: 0.4285278	total: 750ms	remaining: 1m 7s
11:	learn: 0.4212893	total: 813ms	remaining: 1m 6s
12:	learn: 0.4147048	total: 854ms	remaining: 1m 4s
13:	learn: 0.4087742	total: 938ms	remaining: 1m 6s
14:	learn: 0.4037510	total: 1.04s	remaining: 1m 8s
15:	learn: 0.3994909	total: 1.13s	remaining: 1m 9s
16:	learn: 0.3962984	total: 1.21s	remaining: 1m 10s
17:	learn: 0.3929614	total: 1.29s	remaining: 1m 10s
18:	learn: 0.3905901	total: 1.37s	remaining: 1m 10s


In [ ]:
#обработка тестового набора данных

In [103]:
test

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,PearceRAAF,11.0,22.1,0.0,NaN,8.2,ESE,57.0,E,E,30.0,22.0,52.0,34.0,1028.5,1024.6,3.0,NaN,14.7,20.7,0
1,Albury,5.6,20.7,0.0,NaN,NaN,ESE,44.0,SE,NNE,7.0,24.0,65.0,42.0,1023.0,1018.1,NaN,NaN,13.9,20.3,0
2,Cobar,9.5,18.4,0.0,1.6,NaN,SW,26.0,WNW,SSW,2.0,15.0,67.0,43.0,1024.7,1021.5,7.0,7.0,11.5,17.9,0
3,Albany,18.4,22.0,0.0,3.8,0.2,NaN,NaN,E,ESE,22.0,19.0,80.0,79.0,1019.5,1017.0,8.0,8.0,20.7,21.4,0
4,MountGambier,10.4,20.7,0.4,3.8,9.0,SSW,43.0,WSW,SW,15.0,26.0,55.0,38.0,1019.5,1022.4,5.0,4.0,15.7,18.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13536,BadgerysCreek,13.1,26.7,0.2,NaN,NaN,W,52.0,N,WNW,13.0,28.0,58.0,35.0,1005.4,1003.9,NaN,NaN,19.4,26.0,0
13537,Richmond,16.4,30.1,0.0,5.6,NaN,WSW,43.0,ENE,WSW,2.0,20.0,79.0,38.0,1003.2,1000.8,NaN,NaN,21.6,29.7,0
13538,Uluru,22.9,32.9,0.0,NaN,NaN,E,52.0,E,ESE,28.0,20.0,23.0,17.0,1018.4,1015.0,NaN,1.0,25.2,31.4,0
13539,Nhil,12.5,34.2,0.0,NaN,NaN,SSE,31.0,E,ESE,15.0,13.0,37.0,13.0,1021.8,1018.7,NaN,NaN,21.1,31.3,0


In [104]:
print('\n-----------------РАЗМЕРНОСТЬ И ТИП ДАННЫХ---------------------------\n')
print(test.info())
print('\n------------------------НАЛИЧИЕ ПУСТЫХ ЗНАЧЕНИЙ-----------------------------\n')
print(test.isna().sum())


-----------------РАЗМЕРНОСТЬ И ТИП ДАННЫХ---------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13541 entries, 0 to 13540
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Location       13541 non-null  object 
 1   MinTemp        13409 non-null  float64
 2   MaxTemp        13418 non-null  float64
 3   Rainfall       13256 non-null  float64
 4   Evaporation    7933 non-null   float64
 5   Sunshine       7330 non-null   float64
 6   WindGustDir    12565 non-null  object 
 7   WindGustSpeed  12573 non-null  float64
 8   WindDir9am     12547 non-null  object 
 9   WindDir3pm     13146 non-null  object 
 10  WindSpeed9am   13365 non-null  float64
 11  WindSpeed3pm   13258 non-null  float64
 12  Humidity9am    13304 non-null  float64
 13  Humidity3pm    13140 non-null  float64
 14  Pressure9am    12159 non-null  float64
 15  Pressure3pm    12160 non-null  float64
 16  Cloud9am       8407 non

In [105]:
test = test.drop(['Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm'], axis = 1)

In [107]:
for loc in locations:
    average_data = []

    for col in columns_nan_train:
        average_data.append(test[test['Location'] == loc][col].mean())
    d = dict(zip(columns_nan_train, average_data))
    test[test['Location'] == loc] = test[test['Location'] == loc].fillna(value = d)

In [108]:
print(test.isna().sum())

Location            0
MinTemp             0
MaxTemp             0
Rainfall            0
WindGustDir       976
WindGustSpeed     567
WindDir9am        994
WindDir3pm        395
WindSpeed9am        0
WindSpeed3pm        0
Humidity9am         0
Humidity3pm         0
Pressure9am      1089
Pressure3pm      1089
Temp9am             0
Temp3pm             0
RainToday           0
dtype: int64


In [111]:
test['WindGustSpeed'] = test['WindGustSpeed'].fillna(test['WindGustSpeed'].mean())
test['Pressure9am'] = test['Pressure9am'].fillna(test['Pressure9am'].mean())
test['Pressure3pm'] = test['Pressure3pm'].fillna(test['Pressure3pm'].mean())

In [112]:
print(test.isna().sum())

Location           0
MinTemp            0
MaxTemp            0
Rainfall           0
WindGustDir      976
WindGustSpeed      0
WindDir9am       994
WindDir3pm       395
WindSpeed9am       0
WindSpeed3pm       0
Humidity9am        0
Humidity3pm        0
Pressure9am        0
Pressure3pm        0
Temp9am            0
Temp3pm            0
RainToday          0
dtype: int64


In [114]:
for loc in locations:
    average_data = []

    for col in 'WindGustDir', 'WindDir9am', 'WindDir3pm':
        unik = test[test['Location'] == loc][col].unique()
        v_c = test[test['Location'] == loc][col].value_counts(sort=False)

        maks = 0
        for i in range(len(v_c)):
            if v_c[i] > v_c[maks]:
                maks = i
    
        average_data.append(unik[maks])
    d = dict(zip(['WindGustDir', 'WindDir9am', 'WindDir3pm'], average_data))
    test[test['Location'] == loc] = test[test['Location'] == loc].fillna(value = d)

In [116]:
l = []
for i in 'WindGustDir', 'WindDir3pm':
    unik = test['WindGustDir'].unique()
    v_c = test['WindGustDir'].value_counts(sort=False)
    maks = 0
    for i in range(len(v_c)):
        if v_c[i] > v_c[maks]:
            maks = i
    
    l.append(unik[maks])

test['WindGustDir'] = test['WindGustDir'].fillna(l[0])
test['WindDir3pm'] = test['WindDir3pm'].fillna(l[1])

In [119]:
print(test.isna().sum())

Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Temp9am          0
Temp3pm          0
RainToday        0
dtype: int64


In [120]:
for col in 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm':
    label = encoder.fit_transform(test[col])
    test[col] = label

In [121]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13541 entries, 0 to 13540
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Location       13541 non-null  int64  
 1   MinTemp        13541 non-null  float64
 2   MaxTemp        13541 non-null  float64
 3   Rainfall       13541 non-null  float64
 4   WindGustDir    13541 non-null  int64  
 5   WindGustSpeed  13541 non-null  float64
 6   WindDir9am     13541 non-null  int64  
 7   WindDir3pm     13541 non-null  int64  
 8   WindSpeed9am   13541 non-null  float64
 9   WindSpeed3pm   13541 non-null  float64
 10  Humidity9am    13541 non-null  float64
 11  Humidity3pm    13541 non-null  float64
 12  Pressure9am    13541 non-null  float64
 13  Pressure3pm    13541 non-null  float64
 14  Temp9am        13541 non-null  float64
 15  Temp3pm        13541 non-null  float64
 16  RainToday      13541 non-null  int64  
dtypes: float64(12), int64(5)
memory usage: 1.8 MB


In [122]:
# предсказываем результат для тестовых данных

In [123]:
res = model.predict(test)

In [131]:
q = [i for i in range(13541)]

In [141]:
result = pd.DataFrame()
result['index'] = q
result['RainTomorrow'] = res

In [142]:
result.to_csv('Dolzhenko_wheather_prediction.csv', index = False)
files.download('Dolzhenko_wheather_prediction.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>